# Kenneth Lent MDA 610 Project 1
## Restaurant Chatbot Program

### Problem Description
The goal is to create an interactive chatbot for a restaurant that allows customers to:
- View the menu
- Place orders
- View order history
- Get help with commands

#### Data Structures
- Menu stored as nested dictionary with categories and items/prices
- Orders stored as dictionary mapping items to quantities 
- Order history saved to text file for persistence

### Key Functions

#### Main Interface
- `display_welcome()`: Shows welcome message and available commands
- `main()`: Main program loop handling user input and command routing

#### Menu Functions  
- `show_menu()`: Displays formatted menu with item numbers
- `get_valid_quantity()`: Input validation for order quantities

#### Order Processing
- `process_order()`: Handles item selection and builds order
- `process_order_with_save()`: Wrapper to save completed orders
- `save_order()`: Writes order details to history file

#### Order History
- `show_history()`: Displays past orders from file

### Program Flow
1. Display welcome message and command menu
2. Get user input and route to appropriate function:
   - Menu display
   - New order process
   - Order history
   - Help/commands
   - Exit
3. For orders:
   - Show menu
   - Get item selections and quantities
   - Calculate total
   - Save to history
   - Show receipt

### Error Handling
- Input validation for menu selections
- Quantity validation
- File I/O error handling
- Invalid command handling

### Data Persistence
- Orders saved to text file
- File includes:
  - Order date/time
  - Items and quantities
  - Order total

In [1]:
# Menu data structure - Dictionary with categories and items
menu = {
    "Appetizers": {
        "Garlic Bread": 5.99,
        "Mozzarella Sticks": 7.99,
        "Caesar Salad": 8.99
    },
    "Main Courses": {
        "Classic Burger": 12.99,
        "Grilled Chicken": 15.99,
        "Pasta Alfredo": 14.99,
        "Margherita Pizza": 13.99
    },
    "Desserts": {
        "Chocolate Cake": 6.99,
        "Ice Cream Sundae": 5.99,
        "Apple Pie": 6.99
    },
    "Beverages": {
        "Soft Drink": 2.99,
        "Coffee": 3.99,
        "Iced Tea": 2.99
    }
}

def display_welcome():
    """Display welcome message and available commands."""

    print("\n=== Welcome to Kenny's Café Chatbot! ===")
    print("How can I help you today?")
    print("\nAvailable commands:")
    print("1 or 'menu' - View our menu")
    print("2 or 'order' - Place a new order")
    print("3 or 'history' - View order history")
    print("4 or 'help' - Show these commands again")
    print("'quit' - Exit the program")
    print("=" * 37)

def show_menu():
    """Display the menu with item numbers and formatted prices."""
    
    print("\n=== Restaurant Menu ===")
    item_number = 1
    
    # Store item numbers and their corresponding names for later reference
    global menu_items
    menu_items = {}
    
    for category, items in menu.items():
        print(f"\n{category}")
        print("-" * len(category))
        
        for item, price in items.items():
            print(f"{item_number}. {item:<20} ${price:.2f}")
            menu_items[item_number] = item
            item_number += 1
    
    print("\n" + "=" * 20)


In [2]:
def get_valid_quantity():
    """Get a valid quantity from user input."""

    while True:
        try:
            quantity = int(input("Enter quantity (or 0 to cancel): "))
            if quantity < 0:
                print("Please enter a positive number.")
                continue
            return quantity
        except ValueError:
            print("Please enter a valid number.")

def process_order():
    """Handle the order process including item selection and quantity."""
    
    order = {}  # Dictionary to store items and quantities
    total = 0.0
    
    show_menu()  # Show menu at the start of item selection
    while True:
        try:
            choice = input("\nEnter item number (or 'done' to finish): ").lower()
            
            if choice == 'done':
                if not order:
                    print("No items in order.")
                    return None
                break
                
            item_num = int(choice)
            if item_num not in menu_items:
                print("Invalid item number. Please try again.")
                continue
                
            quantity = get_valid_quantity()
            if quantity == 0:
                continue
                
            # Find the item price from the menu
            item_name = menu_items[item_num]
            for category in menu.values():
                if item_name in category:
                    price = category[item_name]
                    break
            
            # Add to or update order
            if item_name in order:
                order[item_name] += quantity
            else:
                order[item_name] = quantity
                
            # Update running total
            total += price * quantity
            
            # Show current order status
            print("\nCurrent Order:")
            print("-" * 30)
            for item, qty in order.items():
                print(f"{item}: {qty}")
            print(f"Current Total: ${total:.2f}")
            
        except ValueError:
            print("Invalid input. Please enter a number or 'done'.")
    
    # Order confirmation
    print("\nOrder Summary:")
    print("-" * 30)
    for item, quantity in order.items():
        for category in menu.values():
            if item in category:
                price = category[item]
                item_total = price * quantity
                print(f"{item:<20} x{quantity:<3} ${item_total:.2f}")
    
    print("-" * 30)
    print(f"Total: ${total:.2f}")
    
    # Confirm order
    while True:
        confirm = input("\nConfirm order? (yes/no): ").lower()
        if confirm in ['yes', 'y']:
            return order, total
        elif confirm in ['no', 'n']:
            return None
        else:
            print("Please enter 'yes' or 'no'.")

In [3]:
from datetime import datetime

def save_order(order, total):
    """Save order details to a simple text file."""

    try:
        # Create receipt with timestamp
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        receipt = f"\nOrder Date: {timestamp}\n"
        receipt += "-" * 30 + "\n"
        
        # Add items
        for item, qty in order.items():
            receipt += f"{item}: {qty}\n"
        
        receipt += "-" * 30 + "\n"
        receipt += f"Total: ${total:.2f}\n"
        receipt += "Thanks for your order!\n"
        
        # Save to file
        with open('order_history.txt', 'a') as f:
            f.write(receipt)
            f.write("\n")
        
        # Show receipt to user
        print(receipt)
        print("Order saved to order_history.txt")
        print("\n" + "=" * 37)  # Visual break
        display_welcome()  # Show menu options again
        
    except Exception as e:
        print("Couldn't save order, but here's your receipt:")
        print(receipt)

def process_order_with_save():
    """Process order and save receipt if successful."""
    
    result = process_order()
    if result:
        order, total = result
        save_order(order, total)

In [ ]:
def show_history():
    """Display recent order history from file."""

    try:
        with open('order_history.txt', 'r') as f:
            history = f.read()
            if history:
                print("\n=== Recent Orders ===")
                print(history)
            else:
                print("\nNo previous orders found.")
    except FileNotFoundError:
        print("\nNo previous orders found.")

def main():
    """Main chatbot loop."""
    
    display_welcome()
    
    while True:
        user_input = input("\nWhat would you like to do? ").lower().strip()
        
        # Handle various input formats
        if user_input in ['1', 'menu', 'show menu', 'view menu']:
            show_menu()
            
        elif user_input in ['2', 'order', 'new order', 'place order']:
            print("\nStarting new order...")
            process_order_with_save()
            
        elif user_input in ['3', 'history', 'orders', 'view history']:
            show_history()
            
        elif user_input in ['4', 'help', 'commands']:
            display_welcome()
            
        elif user_input in ['quit', 'exit', 'bye', 'goodbye']:
            print("\nThank you for visiting Kenny's Café! Goodbye!")
            break
            
        else:
            print("\nI didn't understand that. Here are the available commands:")
            print("1 or 'menu' - View our menu")
            print("2 or 'order' - Place a new order")
            print("3 or 'history' - View order history")
            print("4 or 'help' - Show all commands")
            print("'quit' - Exit the program")


In [5]:
# sample run 1
main()



=== Welcome to Kenny's Café Chatbot! ===
How can I help you today?

Available commands:
1 or 'menu' - View our menu
2 or 'order' - Place a new order
3 or 'history' - View order history
4 or 'help' - Show these commands again
'quit' - Exit the program

=== Restaurant Menu ===

Appetizers
----------
1. Garlic Bread         $5.99
2. Mozzarella Sticks    $7.99
3. Caesar Salad         $8.99

Main Courses
------------
4. Classic Burger       $12.99
5. Grilled Chicken      $15.99
6. Pasta Alfredo        $14.99
7. Margherita Pizza     $13.99

Desserts
--------
8. Chocolate Cake       $6.99
9. Ice Cream Sundae     $5.99
10. Apple Pie            $6.99

Beverages
---------
11. Soft Drink           $2.99
12. Coffee               $3.99
13. Iced Tea             $2.99


=== Recent Orders ===

Order Date: 2024-11-10 12:31:11
------------------------------
Garlic Bread: 1
Grilled Chicken: 1
Iced Tea: 1
------------------------------
Total: $24.97
Thanks for your order!


Order Date: 2024-11-10 12:33:4

In [6]:
# sample run 2
main()




=== Welcome to Kenny's Café Chatbot! ===
How can I help you today?

Available commands:
1 or 'menu' - View our menu
2 or 'order' - Place a new order
3 or 'history' - View order history
4 or 'help' - Show these commands again
'quit' - Exit the program

Starting new order...

=== Restaurant Menu ===

Appetizers
----------
1. Garlic Bread         $5.99
2. Mozzarella Sticks    $7.99
3. Caesar Salad         $8.99

Main Courses
------------
4. Classic Burger       $12.99
5. Grilled Chicken      $15.99
6. Pasta Alfredo        $14.99
7. Margherita Pizza     $13.99

Desserts
--------
8. Chocolate Cake       $6.99
9. Ice Cream Sundae     $5.99
10. Apple Pie            $6.99

Beverages
---------
11. Soft Drink           $2.99
12. Coffee               $3.99
13. Iced Tea             $2.99


Current Order:
------------------------------
Soft Drink: 2
Current Total: $5.98

Current Order:
------------------------------
Soft Drink: 2
Garlic Bread: 1
Current Total: $11.97

Current Order:
--------------

In [7]:
# sample run 3
main()



=== Welcome to Kenny's Café Chatbot! ===
How can I help you today?

Available commands:
1 or 'menu' - View our menu
2 or 'order' - Place a new order
3 or 'history' - View order history
4 or 'help' - Show these commands again
'quit' - Exit the program

=== Recent Orders ===

Order Date: 2024-11-10 12:31:11
------------------------------
Garlic Bread: 1
Grilled Chicken: 1
Iced Tea: 1
------------------------------
Total: $24.97
Thanks for your order!


Order Date: 2024-11-10 12:33:43
------------------------------
Coffee: 1
------------------------------
Total: $3.99
Thanks for your order!


Order Date: 2024-11-10 12:36:33
------------------------------
Coffee: 1
------------------------------
Total: $3.99
Thanks for your order!


Order Date: 2024-11-10 12:39:00
------------------------------
Soft Drink: 2
Garlic Bread: 1
Classic Burger: 1
------------------------------
Total: $24.96
Thanks for your order!



=== Recent Orders ===

Order Date: 2024-11-10 12:31:11
-----------------------